In [ ]:
# default_exp preprocessing.clean

# preprocessing.clean

> Functions to split the raw FHIR dataset, clean and save for further processing & vocab creation.

#hide

[Data Dictionary - Listing Condition Codes](https://docs.google.com/spreadsheets/d/17lR6rpCnSg2B5sLYQjz6yLkwPrK3UgeSn8iE83tZ54Y/edit?usp=sharing)

## Data

- [Health and Economic Costs of Chronic Diseases - CDC](https://www.cdc.gov/chronicdisease/about/costs/index.htm)

### Synthea
- [Synthea - Wiki](https://github.com/synthetichealth/synthea/wiki)
    - contains details about what the project is and how to get started and generate the data.
- [Synthea Data Dictionary](https://github.com/synthetichealth/synthea/wiki/CSV-File-Data-Dictionary)

#### Data Generation
- For our purposes, once Synthea is set up, the following scripts generate the data. 
- Its important to record the run dates, we will use this during preprocessing.

`./run_synthea -s 12345 -p 10000`
- run date: 12-19-2019
- {alive=10000, dead=1076}
- raw - 1.5GB

`./run_synthea -s 54321 -p 20000`
- run date: 11-5-2020
- {alive=20000, dead=2195}
- csv - 2.9GB

`./run_synthea -s 12345 -p 100000`
- run date: 4-4-2020
- {alive=100000, dead=10872}
- csv - 14.6GB

### Folder Structure
- Choose a location to store all Synthea-generated data (
    - for example `~/mydatastore`
- Copy data generated by Synthea into this specific folder structure 
    - for 10K data - `mydatastore/datasets/synthea/10K/raw_original`
- Then create a symbolic link called `datasets` in the current working folder pointing to the datasets folder in the datastore 
    - `ln -sfn ~/mydatastore/datasets datasets`

In [ ]:
#hide
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:85% !important; }</style>"))

In [ ]:
#hide
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
#export
from fastai.imports import *
from fastai import *
from datetime import date

In [ ]:
#hide
assert torch.cuda.is_available() == True
assert torch.backends.cudnn.enabled == True

In [ ]:
#hide
# cuda0 = torch.device('cuda:0')
device = torch.device('cuda',0)
torch.cuda.device(device)

In [ ]:
#hide
from nbdev.showdoc import *

#hide

## Global Variables

In [ ]:
#export
PATH = "."
PATH_1K = f'{PATH}/datasets/synthea/1K'
PATH_10K = f'{PATH}/datasets/synthea/10K'
PATH_20K = f'{PATH}/datasets/synthea/20K'
PATH_100K = f'{PATH}/datasets/synthea/100K'
FILENAMES = ['patients', 'observations', 'allergies', 'careplans', 'medications', 'imaging_studies', 'procedures', 'conditions', 'immunizations']
SYNTHEA_DATAGEN_DATES = dict({'1K':'11-16-2018', '10K':'12-19-2019', '20K':'11-5-2020', '100K':'4-4-2020', '250K':'11-16-2018'})
CONDITIONS = dict({'diabetes':'44054006||START', 'stroke':'230690007||START', 'alzheimers':'26929004||START', 'coronary_heart':'53741008||START'})
LABELS = ['diabetes', 'stroke', 'alzheimers', 'coronaryheart']
CUTOFF_AGE = 20
LOG_NUMERICALIZE_EXCEP = True

## Split 

In [ ]:
os.listdir(f'{PATH_1K}/raw_original')

['patients.csv',
 'observations.csv',
 'allergies.csv',
 'careplans.csv',
 'medications.csv',
 'imaging_studies.csv',
 'procedures.csv',
 'conditions.csv',
 'encounters.csv',
 'immunizations.csv']

In [ ]:
#export
def read_raw_ehrdata(path, csv_names = FILENAMES):
    '''Read raw EHR data'''
    dfs = [pd.read_csv(f'{path}/{fname}.csv', low_memory=False) for fname in csv_names]
    return dfs    

In [ ]:
dfs = read_raw_ehrdata(f'{PATH_1K}/raw_original')

In [ ]:
patients, observations, allergies, careplans, medications, imaging_studies, procedures, conditions, immunizations = dfs

In [ ]:
#export
def split_patients(patients, valid_pct=0.2, test_pct=0.2, random_state=1234):
    '''Split the patients dataframe'''
    train_pct = 1 - (valid_pct + test_pct)
    print(f'Splits:: train: {train_pct}, valid: {valid_pct}, test: {test_pct}')
    patients = patients.sample(frac=1, random_state=random_state).reset_index(drop=True)
    return np.split(patients, [int(train_pct*len(patients)), int((train_pct+valid_pct)*len(patients))])

In [ ]:
train, valid, test = split_patients(patients, .2,.1)

Splits:: train: 0.7, valid: 0.2, test: 0.1


In [ ]:
len(patients), len(train), len(valid), len(test)

(1108, 775, 222, 111)

In [ ]:
assert len(patients) == len(train)+len(valid)+len(test)

In [ ]:
#export
def split_ehr_dataset(path, valid_pct=0.2, test_pct=0.2, random_state=1234):
    '''Split EHR dataset into train, valid, test and save'''

    train_dfs, valid_dfs, test_dfs = [],[],[]
    
    dfs = read_raw_ehrdata(f'{path}/raw_original')
    train_pt, valid_pt, test_pt = split_patients(dfs[0], valid_pct, test_pct, random_state)
    train_dfs.append(train_pt)
    valid_dfs.append(valid_pt)
    test_dfs.append(test_pt)
    print(f'Split {FILENAMES[0]} into:: Train: {len(train_pt)}, Valid: {len(valid_pt)}, Test: {len(test_pt)} -- Total before split: {len(dfs[0])}')
    
    for df, name in zip(dfs[1:], FILENAMES[1:]):
        df = df.set_index('PATIENT')
        df_train = df.loc[df.index.intersection(train_pt['ID']).unique()]
        df_valid = df.loc[df.index.intersection(valid_pt['ID']).unique()]
        df_test = df.loc[df.index.intersection(test_pt['ID']).unique()]
        assert len(df) == len(df_train)+len(df_valid)+len(df_test),f'Split failed {name}: {len(df)} != {len(df_train)}+{len(df_valid)}+{len(df_test)}'
        train_dfs.append(df_train.reset_index())
        valid_dfs.append(df_valid.reset_index())
        test_dfs.append(df_test.reset_index())

    
    for split in ['train', 'valid', 'test']:
        d = Path(f'{path}/raw_split/{split}')
        d.mkdir(parents=True, exist_ok=True)
        
        if split == 'train':
            for df, name in zip(train_dfs, FILENAMES):
                df.to_csv(f'{d}/{name}.csv', index=False)
            print(f'Saved train data to {d}')
        
        if split == 'valid':
            for df, name in zip(valid_dfs, FILENAMES):
                df.to_csv(f'{d}/{name}.csv', index=False)
            print(f'Saved valid data to {d}')
    
        if split == 'test':
            for df, name in zip(test_dfs, FILENAMES):
                df.to_csv(f'{d}/{name}.csv', index=False)
            print(f'Saved test data to {d}')

In [ ]:
split_ehr_dataset(PATH_1K) #will use default values for split percents

Splits:: train: 0.6, valid: 0.2, test: 0.2
Split patients into:: Train: 664, Valid: 222, Test: 222 -- Total before split: 1108
Saved train data to datasets/synthea/1K/raw_split/train
Saved valid data to datasets/synthea/1K/raw_split/valid
Saved test data to datasets/synthea/1K/raw_split/test


## Clean

In [ ]:
#export
def cleanup_pts(pts, is_train, today=None):
    '''Clean patients df'''
    pts.drop(columns=['SSN','DRIVERS','PASSPORT','PREFIX','FIRST','LAST','SUFFIX','MAIDEN','ADDRESS'], inplace=True)
    pts.rename(str.lower, axis='columns', inplace=True)
    pts.rename(columns={"id":"patient"}, inplace=True)
    pts = pts.astype({'birthdate':'datetime64'}) 
    pts['zip'] = pts['zip'].fillna(0.0).astype(int)    
    if today == None: today = pd.Timestamp.today()
    else            : today = pd.to_datetime(today)
    pts['age_now_days'] = pts['birthdate'].apply(lambda bday: (today-bday).days)
    
    pts.set_index('patient', inplace=True)
    pt_demographics = pts.loc[:,['birthdate', 'marital', 'race', 'ethnicity', 'gender', 'birthplace', 'city', 'state', 'zip', 'age_now_days']] 
    patients = pts.loc[:,['birthdate']]
    if is_train: pt_codes = pt_demographics.reset_index(drop=True)
    
    return [patients, pt_demographics, pt_codes] if is_train else [patients, pt_demographics]

In [ ]:
#export
def cleanup_obs(obs, is_train):
    '''Clean observations df'''
    obs.UNITS.fillna('xxxnan', inplace=True)
    obs.dropna(subset=['VALUE'], inplace=True)
    obs.rename(columns={"DATE":"date", "PATIENT":"patient", "CODE":"orig_code", \
                                 "DESCRIPTION":"desc", "VALUE":"value", "UNITS":"units", "TYPE":"type"}, inplace=True)
    obs['code'] = obs['orig_code'].str.cat(obs[['value', 'units', 'type']].astype(str), sep='||')

    if is_train: obs_codes = obs.loc[:, ['orig_code', 'desc', 'value', 'units', 'type']]
    
    obs = obs.loc[:, ['patient', 'date', 'code']]
    obs = obs.astype({'date':'datetime64'})
    obs.set_index('patient', inplace=True)
    
    return [obs, obs_codes] if is_train else [obs]

- Drops rows with null in the `VALUE` column
- Creates a new `code` column with a concatenation of `code`, `value`, `units` and `type`

**Vocab creation for Observations**

**For `numeric`**
```
for 'numeric'
    get unique 'codes'
    for each unique code
        get unique 'units'
            for each unique unit
                bucketize 'values'
                create vocab entry for each 'bucket' -- code||value_bucket||units
```
**For `text`**
```
for 'text'
    get unique 'codes'
    for each unique code
        get unique 'units' #this will be null
            for each unique unit
                get unique 'values'
                create vocab entry for each -- code||value||units
```

In [ ]:
#export
def cleanup_algs(allergies, is_train):
    '''Clean allergies df'''
    allergies.drop(columns=['ENCOUNTER'], inplace=True)
    
    stops = pd.DataFrame(allergies.loc[allergies['STOP'].notnull(),:])
    allergies['CODE'] = allergies['CODE'].apply(lambda x: f'{str(x)}||START')
    stops['CODE'] = stops['CODE'].apply(lambda x: f'{str(x)}||STOP')
    allergies.drop(columns=['STOP'], inplace=True)
    stops.drop(columns=['START'], inplace=True)
    allergies.rename(columns={"START":"date", "PATIENT":"patient", "CODE":"code", "DESCRIPTION":"desc"}, inplace=True)
    stops.rename(columns={"STOP":"date", "PATIENT":"patient", "CODE":"code", "DESCRIPTION":"desc"}, inplace=True)
    allergies = allergies.append(stops, ignore_index=True)
    
    if is_train: alg_codes = allergies.loc[:, ['code', 'desc']]
        
    allergies.drop(columns=['desc'], inplace=True)
    allergies = allergies.astype({'date':'datetime64'})
    allergies.set_index('patient', inplace=True)
    return [allergies, alg_codes] if is_train else [allergies]

In [ ]:
#export
def cleanup_crpls(careplans, is_train):
    '''Clean careplans df'''
    careplans.drop(columns=['ID', 'ENCOUNTER', 'REASONCODE', 'REASONDESCRIPTION'], inplace=True)
    
    stops = pd.DataFrame(careplans.loc[careplans['STOP'].notnull(),:])
    careplans['CODE'] = careplans['CODE'].apply(lambda x: f'{str(x)}||START')
    stops['CODE'] = stops['CODE'].apply(lambda x: f'{str(x)}||STOP')
    careplans.drop(columns=['STOP'], inplace=True)
    stops.drop(columns=['START'], inplace=True)
    careplans.rename(columns={"START":"date", "PATIENT":"patient", "CODE":"code", "DESCRIPTION":"desc"}, inplace=True)
    stops.rename(columns={"STOP":"date", "PATIENT":"patient", "CODE":"code", "DESCRIPTION":"desc"}, inplace=True)
    careplans = careplans.append(stops, ignore_index=True)
    
    if is_train: crpl_codes = careplans.loc[:, ['code', 'desc']]

    careplans.drop(columns=['desc'], inplace=True)
    careplans = careplans.astype({'date':'datetime64'})
    careplans.set_index('patient', inplace=True)
    return [careplans, crpl_codes] if is_train else [careplans]

In [ ]:
#export
def cleanup_meds(medications, is_train):
    '''Clean `medications` df'''
    medications.drop(columns=['ENCOUNTER', 'COST', 'DISPENSES', 'TOTALCOST', 'REASONCODE', 'REASONDESCRIPTION'], inplace=True)
    stops = pd.DataFrame(medications.loc[medications['STOP'].notnull(),:])
    medications['CODE'] = medications['CODE'].apply(lambda x: f'{str(x)}||START')
    stops['CODE'] = stops['CODE'].apply(lambda x: f'{str(x)}||STOP')
    medications.drop(columns=['STOP'], inplace=True)
    stops.drop(columns=['START'], inplace=True)
    medications.rename(columns={"START":"date", "PATIENT":"patient", "CODE":"code", "DESCRIPTION":"desc"}, inplace=True)
    stops.rename(columns={"STOP":"date", "PATIENT":"patient", "CODE":"code", "DESCRIPTION":"desc"}, inplace=True)
    medications = medications.append(stops, ignore_index=True)
    
    if is_train: med_codes = medications.loc[:, ['code', 'desc']]

    medications.drop(columns=['desc'], inplace=True)
    medications = medications.astype({'date':'datetime64'})
    medications.set_index('patient', inplace=True)
    return [medications, med_codes] if is_train else [medications]

In [ ]:
#export
def cleanup_img(imaging_studies, is_train):
    '''Clean `imaging` df'''
    imaging_studies.rename(columns={"DATE":"date", "PATIENT":"patient", "BODYSITE_CODE":"code", "BODYSITE_DESCRIPTION":"desc"}, inplace=True)

    if is_train: img_codes = imaging_studies.loc[:, ['code', 'desc']]
        
    imaging_studies = imaging_studies.loc[:, ['patient', 'date', 'code']]
    imaging_studies = imaging_studies.astype({'date':'datetime64'})
    imaging_studies.set_index('patient', inplace=True)
    return [imaging_studies, img_codes] if is_train else [imaging_studies]

In [ ]:
#export
def cleanup_procs(procedures, is_train):
    '''Clean `procedures` df'''
    procedures.rename(columns={"DATE":"date", "PATIENT":"patient", "CODE":"code", "DESCRIPTION":"desc"}, inplace=True)

    if is_train: proc_codes = procedures.loc[:, ['code', 'desc']]
    
    procedures = procedures.loc[:, ['patient', 'date', 'code']]
    procedures = procedures.astype({'date':'datetime64'})
    procedures.set_index('patient', inplace=True)
    return [procedures, proc_codes] if is_train else [procedures]

In [ ]:
#export
def cleanup_cnds(conditions, is_train):
    '''Clean `conditions` df'''
    conditions.drop(columns=['ENCOUNTER'], inplace=True)
    stops = pd.DataFrame(conditions.loc[conditions['STOP'].notnull(),:])
    conditions['CODE'] = conditions['CODE'].apply(lambda x: f'{str(x)}||START')
    stops['CODE'] = stops['CODE'].apply(lambda x: f'{str(x)}||STOP')
    conditions.drop(columns=['STOP'], inplace=True)
    stops.drop(columns=['START'], inplace=True)
    conditions.rename(columns={"START":"date", "PATIENT":"patient", "CODE":"code", "DESCRIPTION":"desc"}, inplace=True)
    stops.rename(columns={"STOP":"date", "PATIENT":"patient", "CODE":"code", "DESCRIPTION":"desc"}, inplace=True)
    conditions = conditions.append(stops, ignore_index=True)
        
    if is_train: cnd_codes = conditions.loc[:, ['code', 'desc']]
        
    conditions.drop(columns=['desc'], inplace=True)
    conditions = conditions.astype({'date':'datetime64'})
    conditions.set_index('patient', inplace=True)
    return [conditions, cnd_codes] if is_train else [conditions]

In [ ]:
#export
def cleanup_immns(immunizations, is_train):
    immunizations.rename(columns={"DATE":"date", "PATIENT":"patient", "CODE":"code", "DESCRIPTION":"desc"}, inplace=True)

    if is_train: imm_codes = immunizations.loc[:, ['code', 'desc']]
        
    immunizations = immunizations.loc[:, ['patient', 'date', 'code']]
    immunizations = immunizations.astype({'date':'datetime64'})
    immunizations.set_index('patient', inplace=True)
    return [immunizations, imm_codes] if is_train else [immunizations]

### Clean all

In [ ]:
#export
def cleanup_dataset(path, is_train, today=None):
    '''Clean all dfs in a split'''
    dfs = read_raw_ehrdata(path)
 
    pt_data   = cleanup_pts(dfs[0],   is_train, today)
    obs_data  = cleanup_obs(dfs[1],   is_train)
    alg_data  = cleanup_algs(dfs[2],  is_train)
    crpl_data = cleanup_crpls(dfs[3], is_train)
    med_data  = cleanup_meds(dfs[4],  is_train)
    img_data  = cleanup_img(dfs[5],   is_train)
    proc_data = cleanup_procs(dfs[6], is_train)
    cnd_data  = cleanup_cnds(dfs[7],  is_train)
    imm_data  = cleanup_immns(dfs[8], is_train)   
    
    data_tables = [pt_data[0], pt_data[1], obs_data[0], alg_data[0], crpl_data[0], med_data[0], img_data[0], proc_data[0], cnd_data[0], imm_data[0]]
    if is_train:
        code_tables = [pt_data[2], obs_data[1], alg_data[1], crpl_data[1], med_data[1], img_data[1], proc_data[1], cnd_data[1], imm_data[1]]
    
    return (data_tables, code_tables) if is_train else (data_tables)

In [ ]:
data_tables, code_tables = cleanup_dataset(f'{PATH_1K}/raw_split/train', is_train=True)

patients, pt_demographics, observations, allergies, \
careplans, medications, imaging_studies, procedures, conditions, immunizations = data_tables

pt_codes, obs_codes, alg_codes, crpl_codes, med_codes, img_codes, proc_codes, cnd_codes, imm_codes = code_tables

In [ ]:
conditions.count()

date    26426
code    26426
dtype: int64

In [ ]:
obs_codes.count()

orig_code    370128
desc         370128
value        370128
units        370128
type         370128
dtype: int64

## Extract Labels (y) & Insert Age

The labels we intend to predict are conditions and must be in the `CONDITIONS` dict
- Adding them to the `patients` df
- And adding the patient's age when the condition was recorded

Also ..
- Inserting patient's age in months and years into each record df

In [ ]:
for key in CONDITIONS.keys():
    print(key,"::",CONDITIONS[key])

diabetes :: 44054006||START
stroke :: 230690007||START
alzheimers :: 26929004||START
coronary_heart :: 53741008||START


In [ ]:
#export
def extract_ys(patients, conditions, cnd_dict=CONDITIONS):
    '''Extract labels from conditions df and add them to patients df with age'''
    for key in cnd_dict.keys():
        patients = patients.merge(conditions[conditions.code==cnd_dict[key]], how='left', left_index=True, right_index=True)
        patients[f'{key}_y'] = patients.code.notna()
        patients[f'{key}_age'] = ((patients.date - patients.birthdate)//np.timedelta64(1,'Y'))
        patients = patients.drop(columns=['date','code'])
    return patients

In [ ]:
tmp_pts = extract_ys(patients, conditions, CONDITIONS)

In [ ]:
tmp_pts.count()

birthdate             664
diabetes_y            664
diabetes_age           37
stroke_y              664
stroke_age             42
alzheimers_y          664
alzheimers_age         18
coronary_heart_y      664
coronary_heart_age     34
dtype: int64

In [ ]:
#export
def insert_age(df, pts_df):
    '''Insert age in years and months into each of the rec dfs'''
    df = df.merge(pts_df, left_index=True, right_index=True)
    df['age']        = (df['date'] - df['birthdate'])//np.timedelta64(1,'Y')
    df['age_months'] = (df['date'] - df['birthdate'])//np.timedelta64(1,'M')
    return df.drop(columns=['date','birthdate'])

## Do All Functions

In [ ]:
#export
def clean_raw_ehrdata(path, valid_pct=0.2, test_pct=0.2, today=None):
    '''Split, clean, preprocess & save raw EHR data'''
    split_ehr_dataset(path, valid_pct, test_pct)
    
    for split in ['train', 'valid', 'test']:
        split_path = f'{path}/raw_split/{split}'
        if split == 'train': data_tables, code_tables = cleanup_dataset(split_path, is_train=True, today=today)
        else               : data_tables = cleanup_dataset(split_path, is_train=False)
        patients, conditions, rec_tables = data_tables[0], data_tables[8], data_tables[2:]
        patients = extract_ys(patients, conditions, CONDITIONS)
        rec_dfs = [insert_age(rec_df, pd.DataFrame(patients.birthdate)) for rec_df in rec_tables]
        
        cleaned_dir = Path(f'{path}/cleaned/{split}')
        cleaned_dir.mkdir(parents=True, exist_ok=True)
    
        for rec_df,name in zip(rec_dfs,FILENAMES[1:]):
            rec_df.to_csv(f'{cleaned_dir}/{name}.csv')
        patients.reset_index(inplace=True)
        patients.to_csv(f'{cleaned_dir}/patients.csv', index_label='indx')
        data_tables[1].to_csv(f'{cleaned_dir}/patient_demographics.csv')
        print(f'Saved cleaned "{split}" data to {cleaned_dir}')
        
        if split == 'train':
            codes_dir = Path(f'{cleaned_dir}/codes')
            codes_dir.mkdir(parents=True, exist_ok=True)
            for code_df,name in zip(code_tables, FILENAMES):
                code_df.to_csv(f'{codes_dir}/code_{name}.csv', index_label='indx')
            print(f'Saved vocab code tables to {codes_dir}')

In [ ]:
#export
def load_cleaned_ehrdata(path):
    '''Load cleaned, age-filtered EHR data'''
    
    csv_names = FILENAMES.copy()
    csv_names.insert(1,'patient_demographics')
    
    train_dfs = [pd.read_csv(f'{path}/cleaned/train/{fname}.csv', low_memory=False, index_col=0) for fname in csv_names]
    valid_dfs = [pd.read_csv(f'{path}/cleaned/valid/{fname}.csv', low_memory=False, index_col=0) for fname in csv_names]
    test_dfs  = [pd.read_csv(f'{path}/cleaned/test/{fname}.csv', low_memory=False, index_col=0) for fname in csv_names]
                             
    return train_dfs, valid_dfs, test_dfs

In [ ]:
#export
def load_ehr_vocabcodes(path):
    '''Load codes for vocabs'''
    
    code_dfs = [pd.read_csv(f'{path}/cleaned/train/codes/code_{fname}.csv', low_memory=False, na_filter=False, index_col=0) for fname in FILENAMES]
                             
    return code_dfs

In [ ]:
#hide
train_dfs, valid_dfs, test_dfs = load_cleaned_ehrdata(PATH_1K)
code_dfs = load_ehr_vocabcodes(PATH_1K)

In [ ]:
#hide
# for df in train_dfs:
#     display(df.head())

In [ ]:
#hide
# for df in code_dfs:
#     display(df.head())

## Export -

In [ ]:
#hide
from nbdev.export import *
notebook2script()

Converted 01_preprocessing_clean.ipynb.
Converted index.ipynb.
